In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from rwanda.src.utils import get_repo_root
from rwanda.src.impute import impute_data
from rwanda.src.reduce import reduce_dimenions


In [ ]:
# Read training data
train = pd.read_csv(f'{get_repo_root()}/data/train.csv')
# Read test data
test = pd.read_csv(f'{get_repo_root()}/data/test.csv')

Make ID and datevar

In [ ]:
# From data frame train, extract from ID_LAT_LON_YEAR_WEEK column ID_LAT_LON as a new column
train['ID_LAT_LON'] = ["_".join(x.split('_')[0:3]) for x in train['ID_LAT_LON_YEAR_WEEK']]
test['ID_LAT_LON'] = ["_".join(x.split('_')[0:3]) for x in test['ID_LAT_LON_YEAR_WEEK']]
# From data frame train, extract from ID_LAT_LON_YEAR_WEEK column YEAR_WEEK as a new column
train["year_week"] = ["_".join(x.split('_')[3:5]) for x in train['ID_LAT_LON_YEAR_WEEK']]
test["year_week"] = ["_".join(x.split('_')[3:5]) for x in test['ID_LAT_LON_YEAR_WEEK']]


Imputation

Dim Reduction

Preprocessing

Simple Approach:
- this just continues to roll out the weighted mean of the last 7 days

In [ ]:
df = train[['ID_LAT_LON', 'year_week', 'emission']]
# Sort train by ID_LAT_LON and year_week
df = df.sort_values(by=['ID_LAT_LON', 'year_week'])

# Group df by ID_LAT_LON and get the last 7 rows
df = df.groupby(['ID_LAT_LON']).tail(7)
# Create a new column with the weighted mean of the last 7 rows
df['emission_7'] = df.groupby(['ID_LAT_LON'])['emission'].transform(lambda x: x.rolling(7, 1).mean())
# Take the last value of the weighted mean
df = df.groupby(['ID_LAT_LON']).tail(1)

# Rename emission_7 to emission
df.drop(["emission"], axis=1, inplace=True)
df = df.rename(columns={'emission_7': 'emission'})

# Merge test and df 1:m on ID_LAT_LON
test = test[['ID_LAT_LON', 'ID_LAT_LON_YEAR_WEEK']]
submission = pd.merge(test, df, on='ID_LAT_LON', how='left')
submission = submission[['ID_LAT_LON_YEAR_WEEK', 'emission']]

# Make submission
submission.loc[submission["emission"] < 0, "emission"] = 0
submission.to_csv("submission.csv", index=False)

Scale year approach